<div style="border-radius: 5px; padding: 1rem; margin-bottom: 1rem">
<img src="https://www.prototypesforhumanity.com/wp-content/uploads/2022/11/LOGO_UTEC_.png" alt="Banner" width="150" />   
 </div>

# Laboratorio 8: Vectorización, Indexación y Reconocimiento de Rostros

> **Prof. Heider Sanchez**  
> **ACLs:** Ana María Accilio, Sebastián Loza

## Introducción

Las imágenes, y específicamente los rostros, representan uno de los tipos más complejos de datos no estructurados, requiriendo técnicas especializadas para su procesamiento y recuperación eficiente.

El manejo de datos biométricos faciales presenta varios retos:
- La necesidad de convertir información visual compleja en representaciones numéricas procesables.
- El almacenamiento eficiente de vectores de alta dimensionalidad (vectores característicos faciales).
- La implementación de búsquedas rápidas y precisas en grandes colecciones de datos.

Este laboratorio aborda estos desafíos explorando diferentes técnicas de indexación vectorial para optimizar la búsqueda de rostros similares. 

El proceso general consiste en:

1. **Vectorización de rostros**: Utilizando la biblioteca `face_recognition`, convertiremos imágenes de rostros en vectores de características (embeddings) de 128 dimensiones que capturan los rasgos faciales distintivos.

2. **Indexación**: Exploraremos diferentes estructuras de indexación para almacenar y buscar eficientemente estos vectores:
   - Búsqueda lineal (fuerza bruta)
   - Índice Multidimensional con GiST
   - Índice de Alta Dimensión con pgvector
   
3. **Búsqueda de similitud**: Implementaremos búsquedas de k vecinos más cercanos (k-NN) para encontrar rostros similares a una imagen de consulta.


## P1. Desempeño del GiST (RTree) en Altas Dimensiones
Vamos empezar el laboratorio evaluando el desempeño del índice multidimensional GIST de PostgreSQL para diferentes dimensiones del vector característico.

- Crear una tabla con dos campos de tipo "cube".
- Poblar la tabla con N puntos aleatorios de D dimensiones en ambos campos.  
  - $N = 10^6$
  - $D = [2,4,6,8,16,32,64]$
- Luego proceder a crear el índice gist sobre uno de ellos.
- Realizar un análisis comparativo de los tiempos de la búsqueda KNN secuencial y la búsqueda KNN indexada para $K=3$.
- Ambas búsquedas, secuencial e indexada, deben usar la misma consulta SQL pero en diferentes campos.
- Capturar los tiempos del plan de ejecución y elabore la tabla de resultados de forma similar a la tabla del apartado anterior.
- Elaborar gráficos comparativos necesarios para responder las siguientes preguntas:
  - ¿A partir de qué valor de N conviene aplicar el RTree en lugar de búsqueda lineal?
  - ¿Hasta qué dimensión D conviene usar el RTree para búsquedas KNN en lugar de la búsqueda lineal?  


#### Tutorial de Uso de GiST en PostgreSQL
```sql
create extension cube;

-- create a table with two vectors
create table vectors (id serial, vector_lineal cube, vector_gist cube);

-- Populate table with random data of 4 dimensions:
insert into vectors(id, vector_lineal)
    select id,
           cube(ARRAY[round(random()*1000),
                      round(random()*1000),
                      round(random()*1000),
                      round(random()*1000)])
      from generate_series(1, 1000) id;
update vectors set vector_gist = vector_lineal;

-- create the index
CREATE INDEX idx_vectorind ON vectors USING gist(vector_gist);

-- KNN using lineal scan
explain analyze
SELECT id, vector_lineal,
       cube_distance(vector_lineal, '(636, 616, 153, 907)') as D
  FROM vectors
 ORDER BY vector_lineal <-> '(636, 616, 153, 907)'
 LIMIT 5;

-- KNN using the GiST index
explain analyze
SELECT id, vector_gist,
       cube_distance(vector_gist, '(636, 616, 153, 907)') as D
  FROM vectors
 ORDER BY vector_gist <-> '(636, 616, 153, 907)'
 LIMIT 5;
```

In [ ]:

r = {2: (63.231, 1.129),
     4: (58.448, 6.541),
     6: (77.356, 18.043),
     8: (74.181, 31.269),
     16: (86.913, 49.812),
     32: (109.925, 138.682),
     50: (119.806, 431.364)}

## P2. Cargar el Dataset de Rostros

### Instalación de dependencias

Antes de comenzar, necesitamos instalar las siguientes librerías en Python:

```bash
# Instalar face_recognition (incluye dlib)
conda install -c conda-forge face_recognition

# Instalar librerías para manipulación de datos y visualización
pip install pandas numpy matplotlib

# Instalar Rtree para indexación espacial
pip install Rtree
```

> **Nota**: La instalación de `face_recognition` puede ser compleja en Windows. Se recomienda usar conda para facilitar la instalación de las dependencias.


### Descarga del dataset
Descargar [LFW Dataset](https://www.kaggle.com/datasets/atulanandjha/lfwpeople/data) y descomprimir en una carpeta localmente.


In [2]:
# Solución Aquí

## P3. Generar los vectores característicos

En esta etapa, convertiremos las imágenes de rostros en vectores numéricos (embeddings) que representan las características faciales. El proceso consiste en:

1. **Extracción de características**: Usando la biblioteca `face_recognition`, cada imagen se procesa para:
   - Detectar la ubicación del rostro
   - Extraer un vector de 128 dimensiones que captura las características faciales distintivas

2. **Almacenamiento**: Los vectores generados se almacenan en PostgreSQL:
   - Evitar reprocesar las imágenes en cada ejecución
   - Permitir la reutilización de los embeddings en diferentes técnicas de búsqueda

La función `generate_face_embeddings()` debe implementar este proceso:
- Recibe una colección de imágenes y el número N de rostros a procesar
- Genera los embeddings faciales usando el modelo pre-entrenado de `face_recognition`
- Guarda los resultados en base de datos.

In [3]:
# Solución Aquí

## P4: Búsqueda KNN Lineal

En este paso implementaremos el método más básico de búsqueda de vecinos más cercanos (k-NN) utilizando una búsqueda lineal:

1. **Procesamiento de consulta**:
   - Cargar la imagen de consulta
   - Extraer su vector de características usando `face_recognition`

2. **Búsqueda de similitud**:
   - Comparar el vector de consulta con cada vector en la colección utilizando el operador `<->`
   - Ordenar los resultados por distancia y retornar los k vectores más cercanos

Aunque este método es simple de implementar, su complejidad es O(n) ya que requiere comparar con todos los vectores de la colección.

In [4]:
# Solución Aquí

## P5. Búsqueda KNN usando GiST de PostgreSQL

En este paso implementaremos la búsqueda KNN utilizando el índice GiST creado para el campo que almacena los vectores caracteristicos del rostro. 

Se espera que esta implementación ofrezca mejor rendimiento que la búsqueda lineal, ya que el R-Tree optimiza la búsqueda reduciendo el espacio de búsqueda usando su estructura jerárquica.

In [5]:
# Solución Aquí

## P6. Búsqueda KNN usando PyVector de PostgreSQL

En este paso implementaremos la búsqueda KNN utilizando PostgreSQL con la extensión `pgvector`, que proporciona soporte nativo para búsquedas de similitud en vectores. El proceso incluye:

1. **Configuración de la base de datos**:
   - Instalar la extensión pgvector con docker:     
     ```bash
     docker run -d --name pgvector -e POSTGRES_PASSWORD=123456 -p 5502:5432  ankane/pgvector
     ```
   - Habilitar la extensión pgvector  
     ```SQL
     CREATE EXTENSION IF NOT EXISTS vector;
     ```
   - Crear una tabla para almacenar los embeddings faciales
     ```SQL
     CREATE TABLE IF NOT EXISTS face_embeddings (
        id SERIAL PRIMARY KEY,
        name TEXT,
        embedding VECTOR(128)  -- Vectores de 128 dimensiones
     );
     ```
   - Crear un índice IVFFlat para optimizar las búsquedas
     ```SQL
     CREATE INDEX IF NOT EXISTS face_embedding_index
        ON face_embeddings USING ivfflat (embedding vector_cosine_ops) WITH (lists = 100);
     ```

2. **Búsqueda de similitud**:
   - Utilizar el operador `<->` de pgvector para calcular la distancia euclidiana, o `<=>` para calcular la distancia coseno.
   - Ejecutar consultas KNN aprovechando el índice vectorial
   - Recuperar los rostros más similares ordenados por distancia
    ```SQL
     SET ivfflat.probes = 10; -- centroides (clusters) visitados durante una búsqueda

     SELECT id, name, embedding <=> '[0.12, 0.34, ..., 0.99]' AS distance
     FROM face_embeddings
     ORDER BY embedding <=> '[0.12, 0.34, ..., 0.99]'
     LIMIT 5;
    ```

Esta implementación es especialmente útil cuando necesitamos integrar la búsqueda de similitud en una aplicación que ya utiliza PostgreSQL como base de datos principal.

In [6]:
# Solución Aquí

# Análisis de resultados

Realice un análisis comparativo de los cuatro métodos de búsqueda implementados considerando:

1. **Rendimiento**:
   - Comparar tiempos de ejecución
   - Crear gráfico comparativo de velocidad
   - Evaluar con diferentes tamaños de dataset

2. **Efectividad**:
   - Calidad de resultados obtenidos
   - Comparación de precisión entre métodos

3. **Conclusiones**:
   - Ventajas/desventajas de cada método
   - Recomendaciones de uso según el escenario

Incluya gráficos que sustenten sus hallazgos.